In [1]:
import chess
import chess.svg
import random
from IPython.display import SVG, display, update_display, clear_output

In [2]:
def createGame(numPowerups):
  board = chess.Board()
  powerupSquares = random.sample(range(64), numPowerups)
  names = []
  for sq in powerupSquares:
    names += [chess.square_name(sq)]
  return board, powerupSquares


In [3]:
def playMove(board, powerupSquares, move):
  try:
    move = board.parse_san(move)
  except:
    print("Illegal Move")
    return board
  board.push(move)
  if move.to_square in powerupSquares:
    powerup = Powerup(board)
    powerup.randPowerup(move.to_square)
    board = powerup.board
  return board
  

    

In [4]:
def convert_to_int(board):
    indices = '♚♛♜♝♞♟·♙♘♗♖♕♔'
    piece_map = board.piece_map()
    ret = [0] * 64
    for key in piece_map.keys():
      ret[key] = indices.index(piece_map[key].unicode_symbol()) - 6
    return ret

In [5]:
class Powerup:
  def __init__(self, board):
    self.board = board

  def randPowerup(self, square):
    numPowerups = 3
    powerup = random.randint(0,numPowerups - 1)
    if powerup == 0:
      print("White swap!")
      self.swap(True)
    elif powerup == 1:
      print("Black swap!")
      self.swap(False)
    elif powerup == 2:
      print("Go apain!")
      self.repeat()
    else:
      print("Replace!")
      self.replace(square)

  def repeat(self):
    self.board.turn = not self.board.turn


  def swap(self, color):
    # color is boolean, True swap two of white's pieces
    # False swap blacks
    if type(color) is not bool:
      print("Color arg must be bool")
    boardInt = convert_to_int(self.board)
    possibleSquares = []
    if color:  # White's turn
      possibleSquares = []
      for i in range(len(boardInt)):
        if boardInt[i] > 0:
          possibleSquares += [i]
    else:
      for i in range(len(boardInt)):
        if boardInt[i] < 0:
          possibleSquares += [i]
    swapPairs = []
    for i in range(len(possibleSquares)):
      for j in range(len(possibleSquares)):
        if i < j and boardInt[possibleSquares[i]] != boardInt[possibleSquares[j]]:
          swapPairs += [[possibleSquares[i], possibleSquares[j]]]
    random.shuffle(swapPairs)
    while(True):
      board2 = self.board.copy()
      pieceMap = self.board.piece_map()
      pieceMap[swapPairs[i][0]] , pieceMap[swapPairs[i][1]] = pieceMap[swapPairs[i][1]] , pieceMap[swapPairs[i][0]]
      board2.set_piece_map(pieceMap)
      if board2.is_valid():
        self.board = board2
        return
      i += 1
    

  def replace(self, square):
    piece_map = self.board.piece_map()
    if square not in piece_map.keys():
      print("Required swap does not have piece on square")
      return
    currType = piece_map[square].piece_type
    if abs(currType) == 6:  # king requested to stop
      print("King cannot swap")
      return
    possibleOutcome = list(range(1, 6))
    if currType < 0: 
      possibleOutcome *= -1
    possibleOutcome.remove(currType)
    random.shuffle(possibleOutcome)
    for i in range(4):
      board2 = self.board.copy()
      piece_map[square].piece_type = possibleOutcome[i]
      board2.set_piece_map(piece_map)
      if board2.is_valid():
        self.board = board2
        return
      piece_map = self.board.piece_map()


In [7]:
board, powerupSquares = createGame(5)
squareSet = chess.SquareSet()
for sq in powerupSquares:
  squareSet.add(sq)
display(SVG(chess.svg.board(board, size = 600, squares = squareSet)), display_id = '0')
while(not board.is_checkmate()):
  move = input("")
  board = playMove(board, powerupSquares, move)
  update_display(SVG(chess.svg.board(board, size = 600, squares = squareSet, flipped = not board.turn)), display_id = '0')
  clear_output(wait = True)

KeyboardInterrupt: ignored